<a href="https://colab.research.google.com/github/GrigoryBartosh/hse08_ip/blob/master/hw3_classic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/My Drive/ip_hw_3'

/content/drive/My Drive/ip_hw_3


In [0]:
import os
import copy
import zipfile
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm

import cv2

from PIL import Image

from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

PATH_DATA = 'tl-signs-hse-itmo-2020-winter.zip'

In [4]:
xs, ys = [], []
with zipfile.ZipFile(PATH_DATA, 'r') as zip_file:
    with zip_file.open('train.csv') as file:
        train_data = pd.read_csv(file)

    for _, x in tqdm(train_data.iterrows()):
        name, lable = x['filename'], x['class_number']
        with zip_file.open(os.path.join('train', 'train', name)) as img_file:
            image = Image.open(img_file).convert('RGB')
        xs += [image]
        ys += [lable]

num_classes = max(ys) + 1

x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.1)

In [0]:
def get_hog(win_size, blockSize, blockStride, cellSize, nbins):
    win_size = (win_size, win_size)
    blockSize = (blockSize, blockSize)
    blockStride = (blockStride, blockStride)
    cellSize = (cellSize, cellSize)
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0
    nlevels = 64
    hog = cv2.HOGDescriptor(win_size, blockSize, blockStride, cellSize, nbins, 
                            derivAperture, winSigma, histogramNormType, 
                            L2HysThreshold, gammaCorrection, nlevels)
    return hog

In [0]:
def calc_hog_desc(hog, images):
    xs = []
    for image in tqdm(images):
        x = np.array(image, dtype=np.uint8)
        x = hog.compute(x).reshape(-1)
        xs += [x]

    return xs

In [0]:
def train_svm(xs, ys):
    clf = svm.SVC(decision_function_shape='ovr', kernel='poly', degree=3)
    clf.fit(xs, ys)
    return clf

In [0]:
def calc_accuracy(svm, xs, ys):
    return (svm.predict(xs) == ys).sum() / len(ys)

In [59]:
hog = get_hog(win_size=16, blockSize=16, blockStride=8, cellSize=8, nbins=9)

x_hog_train = calc_hog_desc(hog, x_train)
x_hog_val = calc_hog_desc(hog, x_val)

clf = train_svm(x_hog_train, y_train)

accuracy_train = calc_accuracy(clf, x_hog_train, y_train)
accuracy_val = calc_accuracy(clf, x_hog_val, y_val)

print(f'Train accuracy = {accuracy_train}')
print(f'Val accuracy = {accuracy_val}')


(900,)


In [0]:
x_test = []
with zipfile.ZipFile(PATH_DATA, 'r') as zip_file:
    image_names = [f[10:] for f in zip_file.namelist() if 'test' in f]

    for name in tqdm(image_names):
        with zip_file.open(os.path.join('test', 'test', name)) as img_file:
            image = Image.open(img_file).convert('RGB')
        x_test += [image]

x_hog_test = calc_hog_desc(hog, x_test)
y_test = clf.predict(x_hog_test)

predictions = {'filename': [], 'class_number': []}
with torch.no_grad():
    for name, y in tqdm(zip(image_names, y_test)):
        predictions['filename'] += [name]
        predictions['class_number'] += [y]

predictions = pd.DataFrame(predictions)
predictions.to_csv('test_classic.csv', index=False)